In [1]:
import numpy as np
import sympy
import csv
from json import loads

lab_number = '2.2.1'


def toCopyString(matrix) -> str:
    return 'matrix(' + str(matrix).replace('Matrix([', '[').replace('])', ']')[1:-1].replace(' ', '') + ')'

def fromCopyString(string: str):
    return loads(string.replace('matrix(', '[').replace(')', ']'))

def получить_матрицы_варианта(number):
    filename = f'./{lab_number}_variants.txt'
    with open(filename, 'r') as file:
        variants = csv.reader(file, delimiter=',')
        variant = list(variants)[number - 1][1].split(';')
        A = fromCopyString(variant[0])
        B = fromCopyString(variant[1])
        L = fromCopyString(variant[2])
    return A, B, L

In [277]:
A, B, L = получить_матрицы_варианта(19)
A = sympy.Matrix(A)
B = sympy.Matrix(B)
L = sympy.Matrix(L)
L

Matrix([
[-1, 0],
[-2, 0],
[ 0, 0]])

In [278]:
def матрица_управляемости(A, B, L):
    матрица = sympy.zeros(3, 3)
    матрица[:, 0] = B 
    матрица[:, 1] = A*B
    матрица[:, 2] = A**2*B
    return матрица

In [279]:
S = матрица_управляемости(A, B, L)
S

Matrix([
[1,  42,  429],
[2, -11, -172],
[3,   2,   55]])

In [280]:
toCopyString(S)

'matrix([1,42,429],[2,-11,-172],[3,2,55])'

In [247]:
def определитель_матрицы_управляемости(S):
    определитель = S.det()
    # print('Да!' if определитель != 0 else 'Нет :(')
    return определитель

In [281]:
print(определитель_матрицы_управляемости(S))

-10680


In [249]:
def матрица_C(L):
    res = sympy.zeros(2, 3)
    if L[0, 0] or L[0, 1]:
        res[0, 0] = 1
        if L[1, 0] or L[1, 1]:
            res[1, 1] = 1
        else:
            res[1, 2] = 1
    elif L[1, 0] or L[1, 1]:
        res[0, 1] = 1
        if L[2, 0] or L[2, 1]:
            res[1, 2] = 1
    elif L[2, 0] or L[2, 1]:
        res[0, 2] = 1
    
    return res

In [282]:
C = матрица_C(L)
C

Matrix([
[1, 0, 0],
[0, 1, 0]])

In [283]:
toCopyString(C)

'matrix([1,0,0],[0,1,0])'

In [349]:
def вектор_d(A, B, L):
    s = sympy.symbols('s')
    tmp = (s*sympy.eye(3) - A).inv()*B
    tmp = tmp.applyfunc(lambda x: sympy.simplify(x))
    highest_denominator = sympy.fraction(tmp[0, 0])[1]
    for i in range(3):
        if not sympy.div(sympy.fraction(tmp[i, 0])[1], highest_denominator)[1]:
            highest_denominator = sympy.fraction(tmp[i, 0])[1]
    # print(highest_denominator)
    return sympy.Matrix(sympy.Poly(highest_denominator).all_coeffs()[:0:-1])

In [331]:
d = вектор_d(A, B, L)
d

s**3 - 21*s**2 + 101*s + 99


Matrix([
[ 99],
[101],
[-21]])

In [332]:
def матрица_g(A, B, L):
    s = sympy.symbols('s')
    tmp = (s*sympy.eye(3) - A).inv()*B
    tmp = tmp.applyfunc(lambda x: sympy.simplify(x))
    # multiply every element in tmp by denominator of tmp[0, 0]
    tmp = tmp.applyfunc(lambda x: sympy.simplify(x*sympy.fraction(tmp[0, 0])[1]))
    tmp = tmp.applyfunc(lambda x: sympy.simplify(x*sympy.fraction(tmp[1, 0])[1]))
    tmp = tmp.applyfunc(lambda x: sympy.simplify(x*sympy.fraction(tmp[2, 0])[1]))
    return tmp

In [333]:
g = матрица_g(A, B, L)
g

Matrix([
[  (s - 11)*(s + 32)],
[2*s**2 - 53*s + 261],
[3*s**2 - 61*s + 316]])

In [334]:
def матрица_L(A, B, L):
    s = sympy.symbols('s')
    tmp = матрица_g(A, B, L)
    res = sympy.zeros(3, 3)
    res[0] = sympy.Poly(tmp[0, 0], s).all_coeffs()[::-1]
    res[1] = sympy.Poly(tmp[1, 0], s).all_coeffs()[::-1]
    res[2] = sympy.Poly(tmp[2, 0], s).all_coeffs()[::-1]
    return res.T

In [335]:
matrix_L = матрица_L(A, B, L)
matrix_L

Matrix([
[-352,  21, 1],
[ 261, -53, 2],
[ 316, -61, 3]])

In [288]:
toCopyString(matrix_L)

'matrix([-352,21,1],[261,-53,2],[316,-61,3])'

In [264]:
def матрица_H(A, B, L):
    s = sympy.symbols('s')
    C = матрица_C(L)
    tmp = (s*sympy.eye(3) - A + B)

$ \alpha \vec{a} = \beta $

$ \alpha = (I_H - E_H E_B^{-1}I_B)(L^T)^{-1} $

In [351]:
def матрица_ограничения(A, B, L):
    res = sympy.zeros(3, 3)
    d = вектор_d(A, B, L)
    b = sympy.zeros(3, 1)
    # print(L)
    lambdas = sympy.Matrix([L[i, 0] + L[i, 1]*sympy.I for i in range(3)])
    # print(lambdas)
    is_first = True
    for i, lmbd in enumerate(lambdas):
        if lmbd:
            if lmbd.is_real:
                res[i] = sympy.Matrix([1, lmbd, lmbd**2])
                b[i] = -lmbd**3
            else:
                if is_first:
                    angle = sympy.atan2(L[0, 1], L[0, 0])
                    module = sympy.sqrt(L[0, 0]**2 + L[0, 1]**2)
                    res[i] = sympy.Matrix([1, module*sympy.cos(angle), module**2*sympy.cos(2*angle)])
                    b[i] = -module**3*sympy.cos(3*angle)
                    is_first = False
                else:
                    angle = sympy.atan2(L[i, 1], L[i, 0])
                    module = sympy.sqrt(L[i, 0]**2 + L[i, 1]**2)
                    res[i] = sympy.Matrix([0, module*sympy.sin(angle), module**2*sympy.sin(2*angle)])
                    b[i] = -module**3*sympy.sin(3*angle)
        else:
            res[i, i] = 1
            b[i] = d[i]
    res = res.T
    
    a1 = sympy.symbols('a_1')
    a2 = sympy.symbols('a_2')
    a3 = sympy.symbols('a_3')

    a = sympy.Matrix([a1, a2, a3])

    # solve system of equations res*a = b
    a = res.inv()*b

    matrix_L = матрица_L(A, B, L)

    return (res.n(), b.n(), a.n())


In [352]:
alpha, b, a = матрица_ограничения(A, B, L)
alpha

Matrix([
[1.0, -1.0, 1.0],
[1.0, -2.0, 4.0],
[  0,    0, 1.0]])

In [353]:
toCopyString(alpha)

'matrix([1.00000000000000,-1.00000000000000,1.00000000000000],[1.00000000000000,-2.00000000000000,4.00000000000000],[0,0,1.00000000000000])'

In [354]:
b

Matrix([
[  1.0],
[  8.0],
[-21.0]])

In [355]:
toCopyString(b)

'matrix([1.00000000000000],[8.00000000000000],[-21.0000000000000])'

In [356]:
a

Matrix([
[-48.0],
[-70.0],
[-21.0]])

In [357]:
toCopyString(a)

'matrix([-48.0000000000000],[-70.0000000000000],[-21.0000000000000])'

In [358]:
def матрица_Iu(A, B, L):
    res = sympy.zeros(2, 3)
    is_first = True
    for i in range(3):
        if L[i, 0] or L[i, 1]:
            if is_first:
                res[0, i] = 1
                is_first = False
            else:
                res[1, i] = 1
    return res

In [359]:
Iu = матрица_Iu(A, B, L)
Iu

Matrix([
[1, 0, 0],
[0, 1, 0]])

In [360]:
toCopyString(Iu)

'matrix([1,0,0],[0,1,0])'

In [361]:
def матрица_Cu(A, B, L):
    res = sympy.zeros(2, 2)
    C_T = матрица_C(L).T
    cur_row = 0
    for i in range(3):
        if C_T[i, 0] or C_T[i, 1]:
            res[cur_row, :] = C_T[i, :]
            cur_row += 1
    return res

In [362]:
Cu = матрица_Cu(A, B, L)
Cu

Matrix([
[1, 0],
[0, 1]])

In [363]:
toCopyString(Cu)

'matrix([1,0],[0,1])'

In [364]:
def матрица_p(A, B, L):
    matrix_L = матрица_L(A, B, L)
    Iu = матрица_Iu(A, B, L)
    Cu = матрица_Cu(A, B, L)
    a = матрица_ограничения(A, B, L)[-1]
    d = вектор_d(A, B, L)
    res = (Cu.T).inv()*Iu*(matrix_L.inv()).T*(a-d)
    return res

In [365]:
p = матрица_p(A, B, L)
p

Matrix([
[2.92696629213483],
[23.6741573033708]])

In [366]:
toCopyString(p)

'matrix([2.92696629213483],[23.6741573033708])'

In [367]:
for var_number in range(1, 31):
    A, B, L = получить_матрицы_варианта(var_number)
    A = sympy.Matrix(A)
    B = sympy.Matrix(B)
    L = sympy.Matrix(L)
    try:
        S                          = матрица_управляемости(A, B, L)
        определитель               = определитель_матрицы_управляемости(S)
        матрица_C_                 = матрица_C(L)
        матрица_L_                 = матрица_L(A, B, L)
        левая_матрица_ограничения,\
        правая_матрица_ограничения,\
        матрица_А                  = матрица_ограничения(A, B, L)
        Iu                         = матрица_Iu(A, B, L)
        Cu                         = матрица_Cu(A, B, L)
        p                          = матрица_p(A, B, L)

        print(var_number)
        with open(f'./results/{var_number}.txt', 'w') as file:
            file.write(toCopyString(S) + '\n')
            file.write(str(определитель) + '\n')
            file.write(toCopyString(матрица_C_) + '\n')
            file.write(toCopyString(матрица_L_) + '\n')
            file.write(toCopyString(левая_матрица_ограничения) + '\n')
            file.write(toCopyString(правая_матрица_ограничения) + '\n')
            file.write(toCopyString(матрица_А) + '\n')
            file.write(toCopyString(Iu) + '\n')
            file.write(toCopyString(Cu) + '\n')
            file.write(toCopyString(p) + '\n')
    except:
        pass
        

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
